# 데이터 결합( merge, join )

#### [ 병합 - merge ] 
#### [ 병합 키 지정 ] 
#### [ 조인을 위한 집합 연산 지정 ]
#### [ 열이름이 겹치는 경우 병합 - suffixes ]

#### [ 데이터 결합 예제 ]


In [1]:
import pandas as pd
import numpy as np

print("pandas ver : ",pd.__version__)
print("numpy ver : ",np.__version__)

import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

pandas ver :  0.24.2
numpy ver :  1.16.4


### [ 병합 - merge ] 

- **`pandas.merge()`**
  - 1:1, n:1, n:n 의 다양한 조인 유형을 구현함
  - `pandas.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)`
- 두개(left, right)의 입력값 사이에 일치하는 하나이상의 열 이름을 찾아 그것을 키로 사용함

___

###### 1:1 조인
- `concat()`, `append()` 와 유사한 형태의 Join

In [2]:
# employee 컬럼을 가지고 있는 Dataframe 선언
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


- `pandas.merge()`를 이용하여 df1, df2를 하나의 Dataframe으로 결합
- **employee** 컬럼을 기준으로 자동으로 조인 하게됨
- **employee** 를 기준으로 데이터 항목들이 정렬되어짐

In [3]:
# 1:1 병합
df3 = pd.merge(left=df1, right=df2)
display('df3')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


___

###### n:1 조인
- 두개의 키 열(컬럼) 중 하나가 중복된 항목을 포함하는 경우 
- employee, group <- 두개의 키열

In [4]:
# 새로운 Dataframe 선언
df4 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
display('df3','df4')

df3
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

df4
         group supervisor
0   Accounting      Carly
1  Engineering      Guido
2           HR      Steve

In [5]:
result = pd.merge(left=df3, right=df4)
display('result')

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


___

###### n:n 조인
- 병합시 left, right 항목의 키 열에 모두 중복 항목이 존재 하는 경우 

In [6]:
# 새로운 Dataframe 선언
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})
display('df1', 'df5')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,group,skills
0,Accounting,math
1,Accounting,spreadsheets
2,Engineering,coding
3,Engineering,linux
4,HR,spreadsheets


In [7]:
result2 = pd.merge(left=df1, right=df2)
display('result2')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [8]:
display('result','df5')

result
  employee        group  hire_date supervisor
0      Bob   Accounting       2008      Carly
1     Jake  Engineering       2012      Guido
2     Lisa  Engineering       2004      Guido
3      Sue           HR       2014      Steve

df5
         group        skills
0   Accounting          math
1   Accounting  spreadsheets
2  Engineering        coding
3  Engineering         linux
4           HR  spreadsheets
5           HR  organization

In [9]:
# group 컬럼을 중심으로 병합됨
result3 = pd.merge(left=result, right=df5)
display('result3')

,employee,group,hire_date,supervisor,skills
0,Bob,Accounting,2008,Carly,math
1,Bob,Accounting,2008,Carly,spreadsheets
2,Jake,Engineering,2012,Guido,coding
3,Jake,Engineering,2012,Guido,linux
4,Lisa,Engineering,2004,Guido,coding
5,Lisa,Engineering,2004,Guido,linux
6,Sue,HR,2014,Steve,spreadsheets
7,Sue,HR,2014,Steve,organization


### [ 병합 키 지정 ] 

###### on 키워드  
- 키이름 지정
- pandas.merge(left, right, how='inner', **on=None**, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
- 키 열의 이름을 명시적으로 지정

In [10]:
display('df1', 'df2', "pd.merge(df1, df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1, df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

- 키로 만들 열의 이름이 두개의 입력데이터에 모두 포함된 키를 사용해야 함
- 입력데이터에 없는 열의 이름을 사용할경우 Keyerror가 발생함

___

###### left_on, right_on 키워드
- pandas.merge(left, right, how='inner', on=None, **left_on=None, right_on=None,** left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
- 두 데이터가 서로 다른 열 이름을 가졌을때 병합 할 수 있음



In [11]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})

In [12]:
display('df1', 'df3')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


- df3데이터의 name의 열(column)이름과 df1데이터의 employee를 키워드로 지정하여 병합 할 수 있음

In [13]:
df_merge = pd.merge(left=df1, right=df3, left_on='employee', right_on='name')
display('df_merge')

,employee,group,name,salary
0,Bob,Accounting,Bob,70000
1,Jake,Engineering,Jake,80000
2,Lisa,Engineering,Lisa,120000
3,Sue,HR,Sue,90000


- 결과를 확인하면 중복된 데이터를 가진 열이 발생 하였으며 drop()메서드를 사용하여 삭제 가능함
- **`DataFrame.drop()`**
  - `DataFrame.drop(self, labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise')`

In [14]:
# column을 지정하여 삭제
df_merge.drop(columns='name')

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


In [15]:
# column or index 이름을 주고 axis 를 지정하여 삭제
# axis = 0 : index, axis = 1 : column을 의미함
df_merge.drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


___

###### left_index, right_index 키워드
- pandas.merge(left, right, how='inner', on=None, left_on=None, right_on=None, **left_index=False, right_index=False**, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)

- **인덱스로 병합** 할때 사용

In [16]:
df1a = df1.set_index('employee')
df2a = df2.set_index('employee')
display('df1a', 'df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


In [17]:
pd.merge(left=df1a, right=df2a, left_index=True, right_index=True)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


- Dataframe은 인덱스 기반으로 조인하는 메서드를 제공함
- **`DataFrame.join(self, other, on=None, how='left', lsuffix='', rsuffix='', sort=False)`**

In [18]:
display('df1a','df2a')

,group
employee,
Bob,Accounting
Jake,Engineering
Lisa,Engineering
Sue,HR
,hire_date
employee,
Lisa,2004
Bob,2008
Jake,2012


In [19]:
df1a.join(df2a)

,group,hire_date
employee,,
Bob,Accounting,2008
Jake,Engineering,2012
Lisa,Engineering,2004
Sue,HR,2014


In [20]:
# 인덱스와 열을 섞어서 원하는 결과를 얻을 수 있음
# left_index ~ right_on || left_on ~ right_index 를 결합 할 수 있음
display('df1a', 'df3')

df1a
                group
employee             
Bob        Accounting
Jake      Engineering
Lisa      Engineering
Sue                HR

df3
   name  salary
0   Bob   70000
1  Jake   80000
2  Lisa  120000
3   Sue   90000

In [21]:
pd.merge(df1a, df3, left_index=True, right_on='name')

,group,name,salary
0,Accounting,Bob,70000
1,Engineering,Jake,80000
2,Engineering,Lisa,120000
3,HR,Sue,90000


### [ 조인을 위한 집합 연산 지정 ]

- 병합을 할때 기본적으로 내부 조인(inner join)으로 연산되어 교집합 형태의 데이터가 병합됨
- pandas.merge(left, right, **how='inner'**, on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
- `how` 키워드를 사용하여 조인을 명시적으로 지정할 수 있음
  - how : {‘left’, ‘right’, ‘outer’, ‘inner’}, default ‘inner
    - **left**: use only keys from left frame, similar to a SQL left outer join; preserve key order.
    - **right**: use only keys from right frame, similar to a SQL right outer join; preserve key order.
    - **outer**: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.
    - **inner**: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.
- 누락된 데이터는 NaN으로 표현됨


In [22]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6','df7')

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread
,name,drink
0,Mary,wine
1,Joseph,beer


In [23]:
pd.merge(left=df6, right=df7)

,name,food,drink
0,Mary,bread,wine


###### how=left
- left 데이터 기준으로 병합

In [24]:
pd.merge(left=df6, right=df7, how='left')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine


###### how=right
- right 데이터 기준으로 병합

In [25]:
pd.merge(left=df6, right=df7, how='right')

,name,food,drink
0,Mary,bread,wine
1,Joseph,NaN,beer


###### how=outer

In [26]:
pd.merge(left=df6, right=df7, how='outer')

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,beer


### [ 열이름이 겹치는 경우 병합 - suffixes ]

- 병합을 위한 두개의 입력데이터가 열(column)이름이 충돌할 경우 `suffixes` 키워드 사용
- pandas.merge(left, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, **suffixes=('_x', '_y')**, copy=True, indicator=False, validate=None)

In [27]:
df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
display('df8', 'df9')

,name,rank
0,Bob,1
1,Jake,2
2,Lisa,3
3,Sue,4
,name,rank
0,Bob,3
1,Jake,1
2,Lisa,4
3,Sue,2


In [28]:
pd.merge(df8, df9, on="name")

,name,rank_x,rank_y
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


In [29]:
pd.merge(df8, df9, on="name", suffixes=["_L", "_R"])

,name,rank_L,rank_R
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


- 동일한 열(column)의 이름에 suffixes로 지정한 문자열을 열 이름에 추가(접미사)하여 구분함

### [ 데이터 결합 예제 ]

- 2010년 미국 주의 인구 및 지역 데이터 세트

- 예제 데이터 위치 

In [30]:
!ls data

state-abbrevs.csv    state-areas.csv      state-population.csv


- 데이터 읽어오기

In [31]:
# 인구 데이터
pop = pd.read_csv('data/state-population.csv')
# 지역 면적 데이터
areas = pd.read_csv('data/state-areas.csv')
# 지역 약자 표시 데이터
abbrevs = pd.read_csv('data/state-abbrevs.csv')

display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

#### pop 데이터와 abbrevs 데이터 병합
- 조건1) pop 데이터의 state/region 열과 abbrevs 데이터의 abbreviation열을 기준으로 병합
- 조건2) 데이터가 누락되지 않고 병합

In [32]:
# left_on, right_on 를 이용하여 기준이 되는 키를 지정
# 누락되는 데이터 없이 결합하기 위하여 how = 'outer' 로 지정함
merged = pd.merge(left=pop, right=abbrevs, how='outer', left_on='state/region', right_on='abbreviation')
display('merged.head()', 'merged.tail()')

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL
,state/region,ages,year,population,state,abbreviation
2539,USA,total,2010,309326295.0,NaN,NaN
2540,USA,under18,2011,73902222.0,NaN,NaN
2541,USA,total,2011,311582564.0,NaN,NaN
2542,USA,under18,2012,73708179.0,NaN,NaN


In [33]:
# 중복되는 열 abbreviation 의 데이터 삭제
merged = merged.drop(columns='abbreviation')
display('merged.head()', 'merged.tail()')

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama
,state/region,ages,year,population,state
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN


###### pop, abbrevs 병합 데이터의 NaN  데이터 확인
1. `DataFrame.isna(self)` 
  - **NaN** 데이터를 **True**로 표현함
  - retrun : Mask of bool values for each element in DataFrame that indicates whether an element is not an NA value.

In [34]:
NanMerged = merged.isna()
display('NanMerged.head()', 'NanMerged.tail()')

,state/region,ages,year,population,state
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
,state/region,ages,year,population,state
2539,False,False,False,False,True
2540,False,False,False,False,True
2541,False,False,False,False,True
2542,False,False,False,False,True


In [35]:
#display('NanMerged')

2. `DataFrame.isnull(self)`
  - **NaN** 데이터를 **True**로 표현함
  - return : Mask of bool values for each element in DataFrame that indicates whether an element is not an NA value.
  - isnull 은 inna와 동일한 기능임( 일종의 별칭임 ) 

In [36]:
# 동일한 함수 inna임
print(pd.isnull)
print(pd.isna)

<function isna at 0x1183b3378>
<function isna at 0x1183b3378>


In [37]:
NanMerged2 = merged.isnull()
display('NanMerged2.head()','NanMerged2.tail()')

,state/region,ages,year,population,state
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
,state/region,ages,year,population,state
2539,False,False,False,False,True
2540,False,False,False,False,True
2541,False,False,False,False,True
2542,False,False,False,False,True


In [38]:
#display('NanMerged')

###### pop, abbrevs 병합 데이터의 NaN  데이터 항목 확인
- `DataFrame.any(self, axis=0, bool_only=None, skipna=True, level=None, **kwargs)`
  - Return whether any element is True, potentially over an axis.
  - axis={0:열, 1:행}
  

In [39]:
# 열을 기준으로 NaN 값이 있는지 확인
merged.isna().any(axis=0)

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [40]:
# 행을 기준으로 Nan 값이 있는지 확인
print(merged.isna().any(axis=1).head())
print()
print(merged.isna().any(axis=1).tail())

0    False
1    False
2    False
3    False
4    False
dtype: bool

2539    True
2540    True
2541    True
2542    True
2543    True
dtype: bool


In [41]:
# population 열의 NaN 데이터가 포함된 항목의 head 확인
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


###### state 열의 NaN 데이터 확인

In [42]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

- 푸에르토리코(PR), 미국(USA)에 대한 항목이 NaN을 포함하고 있음

- PR, USA에 대한 abbreviation 값을 채우기

In [43]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
# NaN 데이터를 채우고 나서 NaN 데이터 확인
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [44]:
display('merged', 'areas')

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama
5,AL,total,2011,4801627.0,Alabama
6,AL,total,2009,4757938.0,Alabama
7,AL,under18,2009,1134192.0,Alabama
8,AL,under18,2013,1111481.0,Alabama
9,AL,total,2013,4833722.0,Alabama


###### state 컬럼을 기준으로 areas 데이터를 병합

In [61]:
final = pd.merge(left=merged, right=areas,on='state', how='left')
display('final.head()','final.tail()')

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
,state/region,ages,year,population,state,area (sq. mi)
2539,USA,total,2010,309326295.0,United States,NaN
2540,USA,under18,2011,73902222.0,United States,NaN
2541,USA,total,2011,311582564.0,United States,NaN
2542,USA,under18,2012,73708179.0,United States,NaN


In [62]:
final.isna().any(axis=0)

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

- area, population, abbreviation 에 NaN 값이 존재함

###### area 열의 NaN 항목 확인

In [63]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

- United States 항목에 NaN 데이터가 포함되어 있음
- NaN 값을 가진 행은 삭제
  - `DataFrame.dropna(self, axis=0, how='any', thresh=None, subset=None, inplace=False)` 
    - inplace = True 일 경우 기존 입력 데이터가 수정됨

In [64]:
a = final.dropna()
a.size

14856

In [65]:
# 기존 데이터의 크기
final.size

15264

In [66]:
# inplace 키워드를 사용하여 원본데이터를 수정함
b = final.dropna(inplace=True)
# inplace = True 를 사용하였기 때문에 b 에 데이터가 리턴하지 않음
b.size

AttributeError: 'NoneType' object has no attribute 'size'

In [67]:
# 수정된 후 크기
final.size

14856